# Setting

In [1]:
!pip install category_encoders
import category_encoders as ce

In [2]:
import lightgbm as lgb

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/train.csv')
df_test_x = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/test.csv')
df_holidays = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/holidays_in_japan.csv')
df_venue = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/venue_information.csv')
df_match = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/match_reports.csv')

In [5]:
df_train_x = df_train.drop(['attendance'], axis=1)
df_train_y = df_train['attendance']

# experience

In [6]:
df_venue.head()

,venue,capacity,address
0,札幌ドーム,38794,北海道札幌市豊平区羊ヶ丘1
1,埼玉スタジアム2002,62010,埼玉県さいたま市緑区中野田500
2,味の素スタジアム,47851,東京都調布市西町376−3
3,町田GIONスタジアム,15320,東京都町田市野津田町2035
4,日産スタジアム,71624,神奈川県横浜市港北区小机町3300


In [7]:
df_train_x.head()

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters
0,9190,2006-03-04,16:04,第1節,第1日,G大阪,浦和,万博記念競技場,晴,8.3,40,NHK総合/J SPORTS(録)
1,9191,2006-03-05,13:00,第1節,第2日,甲府,清水,山梨県小瀬スポーツ公園陸上競技場,晴,12.9,28,山梨放送/テレビ静岡(録)/J SPORTS(録)
2,9192,2006-03-05,13:35,第1節,第2日,FC東京,大分,味の素スタジアム,晴,12.1,35,BS-i/MXテレビ(録)/J SPORTS(録)
3,9193,2006-03-05,14:04,第1節,第2日,磐田,福岡,静岡スタジアムエコパ,晴,11.6,42,J SPORTS
4,9194,2006-03-05,14:04,第1節,第2日,名古屋,C大阪,名古屋市瑞穂陸上競技場,晴,13.1,32,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録)


In [8]:
def extract_prefecture(address):
    if '都' in address:
        return address[:address.find('都') + 1]
    elif '道' in address:
        return address[:address.find('道') + 1]
    elif '府' in address:
        return address[:address.find('府') + 1]
    elif '県' in address:
        return address[:address.find('県') + 1]
    else:
        return None


In [9]:
df_venue['prefecture'] = df_venue['address'].apply(extract_prefecture)
df_venue.head()

,venue,capacity,address,prefecture
0,札幌ドーム,38794,北海道札幌市豊平区羊ヶ丘1,北海道
1,埼玉スタジアム2002,62010,埼玉県さいたま市緑区中野田500,埼玉県
2,味の素スタジアム,47851,東京都調布市西町376−3,東京都
3,町田GIONスタジアム,15320,東京都町田市野津田町2035,東京都
4,日産スタジアム,71624,神奈川県横浜市港北区小机町3300,神奈川県


In [11]:
df_train_x_marged = pd.merge(df_train_x, df_match, on='id', how='left')
df_train_x_marged.head()

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,...,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11
0,9190,2006-03-04,16:04,第1節,第1日,G大阪,浦和,万博記念競技場,晴,8.3,...,2 坪井慶介 DF,4 田中マルクス闘莉王 DF,20 堀之内聖 DF,6 山田暢久 MF,17 長谷部誠 MF,13 鈴木啓太 MF,8 三都主アレサンドロ MF,18 小野伸二 MF,10 ポンテ FW,21 ワシントン FW
1,9191,2006-03-05,13:00,第1節,第2日,甲府,清水,山梨県小瀬スポーツ公園陸上競技場,晴,12.9,...,25 市川大祐 DF,26 青山直晃 DF,4 高木和道 DF,3 山西尊裕 DF,13 兵働昭弘 MF,7 伊東輝悦 MF,16 枝村匠馬 MF,10 藤本淳吾 MF,18 チョジェジン FW,17 マルキーニョス FW
2,9192,2006-03-05,13:35,第1節,第2日,FC東京,大分,味の素スタジアム,晴,12.1,...,2 三木隆司 DF,4 深谷友基 DF,22 上本大海 DF,5 エジミウソン MF,11 トゥーリオ MF,6 梅田高志 MF,17 根本裕一 MF,26 内村圭宏 MF,13 高松大樹 FW,9 オズマール FW
3,9193,2006-03-05,14:04,第1節,第2日,磐田,福岡,静岡スタジアムエコパ,晴,11.6,...,22 中村北斗 DF,4 金古聖司 DF,5 千代反田充 DF,3 アレックス DF,7 宮崎光平 MF,6 布部陽功 MF,8 ホベルト MF,14 古賀誠史 MF,27 田中佑昌 FW,36 グラウシオ FW
4,9194,2006-03-05,14:04,第1節,第2日,名古屋,C大阪,名古屋市瑞穂陸上競技場,晴,13.1,...,2 ブルーノクアドロス DF,5 前田和哉 DF,3 柳本啓成 DF,4 河村崇大 MF,23 下村東美 MF,10 ピンゴ MF,6 ゼカルロス MF,8 森島寛晃 FW,20 西澤明訓 FW,9 古橋達弥 FW


In [12]:
df_train_x_marged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3672 entries, 0 to 3671
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  3672 non-null   int64  
 1   match_date          3672 non-null   object 
 2   kick_off_time       3672 non-null   object 
 3   section             3672 non-null   object 
 4   round               3672 non-null   object 
 5   home_team           3672 non-null   object 
 6   away_team           3672 non-null   object 
 7   venue               3672 non-null   object 
 8   weather             3672 non-null   object 
 9   temperature         3672 non-null   float64
 10  humidity            3672 non-null   int64  
 11  broadcasters        3672 non-null   object 
 12  home_team_player11  3672 non-null   object 
 13  home_team_player10  3672 non-null   object 
 14  home_team_player9   3672 non-null   object 
 15  home_team_player8   3672 non-null   object 
 16  home_t